# 压缩

![](img/img_1.png)

In [9]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i + 1}:\n\n" + d.page_content for i, d in enumerate(docs)]))


In [8]:
# !mkdir data
!wget https://raw.githubusercontent.com/run-llama/llama_index/main/README.md -O ../../file/paul_graham_essay.txt

zsh:1: no matches found: https://help.aliyun.com/zh/dashscope/developer-reference/tongyi-qianwen-7b-14b-72b-api-detailes?spm=5176.28197632.0.0.11867e06vodKWB
zsh:1: command not found: -O


In [4]:
# !which wget
# !brew install wget

/opt/homebrew/bin/wget


In [16]:
question = "Qwen2 是什么?"


In [12]:
# CloudflareWorkersAI
from dotenv import load_dotenv
import os
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')

print(account_id)
print(api_token)

import getpass

model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(account_id=account_id, api_token=api_token, model=model)

# 最新的Embedding方式
# cloudflare_workersai
from langchain_community.embeddings.cloudflare_workersai import (
    CloudflareWorkersAIEmbeddings,
)

# //维度是：384
embeddings = CloudflareWorkersAIEmbeddings(
    account_id=account_id,
    api_token=api_token,
    model_name="@cf/baai/bge-small-en-v1.5",
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn


In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import WebBaseLoader
from langchain.vectorstores import Chroma

# Load blog post
loader = WebBaseLoader(
    "https://help.aliyun.com/zh/dashscope/developer-reference/tongyi-qianwen-7b-14b-72b-api-detailes?spm=5176.28197632.0.0.11867e06vodKWB&disableWebsiteRedirect=true")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

retriever = Chroma.from_documents(splits, embeddings, collection_name="example").as_retriever()

In [18]:
# 普通检索，未压缩
docs = retriever.get_relevant_documents(question)
pretty_print_docs(docs)

Document 1:

大语言模型说明 支持的领域 / 任务：aigc Qwen2Qwen2是Qwen开源大语言模型的新系列。参数范围包括0.5B到72B，包括Mixture-of-Experts模型。与最先进的开源语言模型（包括之前发布的 Qwen1.5）相比，Qwen2在一系列针对语言理解、语言生成、多语言能力、编码、数学、推理等的基准测试中总体上超越了大多数开源模型，并表现出与专有模型的竞争力。Qwen2增⼤了上下⽂⻓度⽀持，最⾼达到128K tokens（Qwen2-72B-Instruct），能够处理大量输入。灵积平台上基于Qwen2开源的0.5B、1.5B、7B、72B和57B-A14B MoE模型的instruct版本，并进行了针对性的推理性能优化，为广大开发者提供便捷的API服务。各个版本均对应魔搭社区开源的各个模型版本，详细参考
----------------------------------------------------------------------------------------------------
Document 2:

大语言模型说明 支持的领域 / 任务：aigc Qwen2Qwen2是Qwen开源大语言模型的新系列。参数范围包括0.5B到72B，包括Mixture-of-Experts模型。与最先进的开源语言模型（包括之前发布的 Qwen1.5）相比，Qwen2在一系列针对语言理解、语言生成、多语言能力、编码、数学、推理等的基准测试中总体上超越了大多数开源模型，并表现出与专有模型的竞争力。Qwen2增⼤了上下⽂⻓度⽀持，最⾼达到128K tokens（Qwen2-72B-Instruct），能够处理大量输入。灵积平台上基于Qwen2开源的0.5B、1.5B、7B、72B和57B-A14B MoE模型的instruct版本，并进行了针对性的推理性能优化，为广大开发者提供便捷的API服务。各个版本均对应魔搭社区开源的各个模型版本，详细参考
----------------------------------------------------------------------------------------------------
Document 3:

2,000。qwen1.5-0.5b-chat通义

In [19]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# 压缩方式1
# LLMChainExtractor：文档压缩器，可以使用LLM链提取文档中的相关部分。
compressor = LLMChainExtractor.from_llm(cf_llm)
# ContextualCompressionRetriever：包装基础检索器并压缩结果的检索器。
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

# 压缩之后
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

Extracted relevant parts:
```
Qwen2是Qwen开源大语言模型的新系列。参数范围包括0.5B到72B，包括Mixture-of-Experts模型。
```
----------------------------------------------------------------------------------------------------
Document 2:

Qwen2 是什么?

Extracted relevant parts: Qwen2 是 Qwen 开源大语言模型的新系列。
----------------------------------------------------------------------------------------------------
Document 3:

NO_OUTPUT

The provided question is "Qwen2 是什么?" and the context does not mention anything about "Qwen2". Therefore, none of the context is relevant to answer the question.


In [20]:
from langchain.retrievers.document_compressors import LLMChainFilter

# 压缩方式2
# LLMChainFilter：过滤器，删除与查询无关的文档。
_filter = LLMChainFilter.from_llm(cf_llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=_filter, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

大语言模型说明 支持的领域 / 任务：aigc Qwen2Qwen2是Qwen开源大语言模型的新系列。参数范围包括0.5B到72B，包括Mixture-of-Experts模型。与最先进的开源语言模型（包括之前发布的 Qwen1.5）相比，Qwen2在一系列针对语言理解、语言生成、多语言能力、编码、数学、推理等的基准测试中总体上超越了大多数开源模型，并表现出与专有模型的竞争力。Qwen2增⼤了上下⽂⻓度⽀持，最⾼达到128K tokens（Qwen2-72B-Instruct），能够处理大量输入。灵积平台上基于Qwen2开源的0.5B、1.5B、7B、72B和57B-A14B MoE模型的instruct版本，并进行了针对性的推理性能优化，为广大开发者提供便捷的API服务。各个版本均对应魔搭社区开源的各个模型版本，详细参考
----------------------------------------------------------------------------------------------------
Document 2:

大语言模型说明 支持的领域 / 任务：aigc Qwen2Qwen2是Qwen开源大语言模型的新系列。参数范围包括0.5B到72B，包括Mixture-of-Experts模型。与最先进的开源语言模型（包括之前发布的 Qwen1.5）相比，Qwen2在一系列针对语言理解、语言生成、多语言能力、编码、数学、推理等的基准测试中总体上超越了大多数开源模型，并表现出与专有模型的竞争力。Qwen2增⼤了上下⽂⻓度⽀持，最⾼达到128K tokens（Qwen2-72B-Instruct），能够处理大量输入。灵积平台上基于Qwen2开源的0.5B、1.5B、7B、72B和57B-A14B MoE模型的instruct版本，并进行了针对性的推理性能优化，为广大开发者提供便捷的API服务。各个版本均对应魔搭社区开源的各个模型版本，详细参考
----------------------------------------------------------------------------------------------------
Document 3:

2,000。qwen1.5-0.5b-chat通义

In [22]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

# 压缩方式3
# EmbeddingsFilter: 一种使用嵌入技术来丢弃与查询无关的文档的文档压缩器。
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.8)
compression_retriever = ContextualCompressionRetriever(base_compressor=embeddings_filter, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

In [26]:
from langchain.document_transformers import EmbeddingsRedundantFilter
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.text_splitter import CharacterTextSplitter

# 压缩方式4
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
# EmbeddingsRedundantFilter：通过比较其嵌入来丢弃冗余文档的过滤器
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
# DocumentCompressorPipeline： 使用 Transformer 管道的文档压缩器。
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [27]:
# 压缩方式5
# ContextualCompressionRetriever：包装基础检索器并压缩结果的检索器。
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline_compressor, base_retriever=retriever)

compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

Document 1:

大语言模型说明 支持的领域 / 任务：aigc Qwen2Qwen2是Qwen开源大语言模型的新系列。参数范围包括0.5B到72B，包括Mixture-of-Experts模型。与最先进的开源语言模型（包括之前发布的 Qwen1.5）相比，Qwen2在一系列针对语言理解、语言生成、多语言能力、编码、数学、推理等的基准测试中总体上超越了大多数开源模型，并表现出与专有模型的竞争力。Qwen2增⼤了上下⽂⻓度⽀持，最⾼达到128K tokens（Qwen2-72B-Instruct），能够处理大量输入。灵积平台上基于Qwen2开源的0.5B、1.5B、7B、72B和57B-A14B MoE模型的instruct版本，并进行了针对性的推理性能优化，为广大开发者提供便捷的API服务。各个版本均对应魔搭社区开源的各个模型版本，详细参考
----------------------------------------------------------------------------------------------------
Document 2:

2,000。qwen1.5-0.5b-chat通义千问1.5对外开源的0.5B规模参数量的经过人类指令对齐的chat模型codeqwen1.5-7b-chat通义千问1.5对外开源的7B规模参数量的经过人类指令对齐的针对代码场景的chat模型模型支持 64,000 tokens上下文，为了保障正常使用和正常输出，API限定用户输入为56,000，输出最大 8,000。qwen-72b-chat通义千问对外开源的72B规模参数量的经过人类指令对齐的chat模型模型支持 32,000 tokens上下文，为了保障正常使用和正常输出，API限定用户输入为30,000，输出最大 2,000。qwen-14b-chat通义千问对外开源的14B规模参数量的经过人类指令对齐的chat模型模型支持 8,000 tokens上下文，为了保障正常使用和正常输出，API限定用户输入为6,000，输出最大
